## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.09,70,100,15.28,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,37.51,56,40,9.19,scattered clouds
2,2,Kaeo,NZ,-35.1000,173.7833,50.32,96,83,12.73,moderate rain
3,3,Hoi An,VN,15.8794,108.3350,87.76,83,98,4.63,overcast clouds
4,4,Winnemucca,US,40.9730,-117.7357,69.12,20,1,6.24,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hoi An,VN,15.8794,108.3350,87.76,83,98,4.63,overcast clouds
4,4,Winnemucca,US,40.9730,-117.7357,69.12,20,1,6.24,clear sky
5,5,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,80.85,77,79,11.90,broken clouds
6,6,Anahuac,MX,27.2333,-100.1500,83.70,63,81,17.20,broken clouds
9,9,Ust-Ishim,RU,57.6935,71.1665,63.77,66,57,6.20,broken clouds
...,...,...,...,...,...,...,...,...,...,...
698,698,Lively,CA,46.4334,-81.1498,66.02,23,0,3.87,clear sky
703,703,Yarada,IN,17.6500,83.2667,88.12,84,40,3.15,scattered clouds
708,708,Santos Dumont,BR,-21.4567,-43.5525,85.60,43,79,1.63,broken clouds
711,711,Gushikawa,JP,26.3544,127.8686,75.20,92,90,19.06,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df[ (preferred_cities_df["City"].isnull())]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#no need to drop rows, but here is the code...
clean_df = preferred_cities_df[ (preferred_cities_df["City"].notnull()) & (preferred_cities_df["City"]!=u'') ]
clean_df = clean_df.reset_index()
del clean_df["index"]

In [7]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
3,Hoi An,VN,87.76,15.8794,108.3350,overcast clouds,
4,Winnemucca,US,69.12,40.9730,-117.7357,clear sky,
5,Sao Jose Da Coroa Grande,BR,80.85,-8.8978,-35.1478,broken clouds,
6,Anahuac,MX,83.70,27.2333,-100.1500,broken clouds,
9,Ust-Ishim,RU,63.77,57.6935,71.1665,broken clouds,
12,Cayenne,GF,84.24,4.9333,-52.3333,broken clouds,
17,Martapura,ID,77.09,-3.4167,114.8500,overcast clouds,
20,Mirante Do Paranapanema,BR,75.13,-22.2919,-51.9064,overcast clouds,
21,Honiara,SB,76.33,-9.4333,159.9500,broken clouds,
23,Santa Maria Da Vitoria,BR,92.44,-13.3947,-44.1886,broken clouds,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # If an error is experienced, skip the city.
    except:
        print(f"Skipping. No Hotels returned for: {city}")
        pass


Skipping. No Hotels returned for: Merauke
Skipping. No Hotels returned for: Yasenskaya
Skipping. No Hotels returned for: Beloha
Skipping. No Hotels returned for: Yaan
Skipping. No Hotels returned for: Chapayevka
Skipping. No Hotels returned for: Avera
Skipping. No Hotels returned for: Abu Samrah
Skipping. No Hotels returned for: Latung
Skipping. No Hotels returned for: Ler
Skipping. No Hotels returned for: Ankazobe
Skipping. No Hotels returned for: Slobozia
Skipping. No Hotels returned for: Maridi
Skipping. No Hotels returned for: Yumen
Skipping. No Hotels returned for: Gazli
Skipping. No Hotels returned for: Nioro
Skipping. No Hotels returned for: Tiznit
Skipping. No Hotels returned for: Sinyavskoye
Skipping. No Hotels returned for: Wawa
Skipping. No Hotels returned for: Saint-Augustin
Skipping. No Hotels returned for: Bukama
Skipping. No Hotels returned for: Taguatinga
Skipping. No Hotels returned for: Uvalde
Skipping. No Hotels returned for: Ouadda
Skipping. No Hotels returned for: 

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))